In [35]:
import re
import string
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [40]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ammar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [41]:
df = pd.read_csv(r'./tweets.csv')
df.head()

,Tweet,Avg
0,"I have to say, Apple has by far the best custo...",2.0
1,iOS 7 is so fricking smooth & beautiful!! #Tha...,2.0
2,LOVE U @APPLE,1.8
3,"Thank you @apple, loving my new iPhone 5S!!!!!...",1.8
4,.@apple has the best customer service. In and ...,1.8


In [42]:
df = df['Tweet']
df = df.astype('str')
df.shape

(1181,)

In [43]:
df.head()

0    I have to say, Apple has by far the best custo...
1    iOS 7 is so fricking smooth & beautiful!! #Tha...
2                                        LOVE U @APPLE
3    Thank you @apple, loving my new iPhone 5S!!!!!...
4    .@apple has the best customer service. In and ...
Name: Tweet, dtype: object

In [44]:
translate_table = dict((ord(char), None) for char in string.punctuation)
stop_words = set(stopwords.words('english'))

In [45]:
# Adding custom stopwords
new_stop_words = ['some', 'like', 'think', 'wow', 'one', 'http', 'web', 'really', 'see', 'watch', 'apple', 'know', 'show'
                 'think', 'click', 'go', 'to', 'great', 'very', 'good', 'many', 'more', 'people', 'made', 'technology', 'tech']
stop_words = stop_words.union(new_stop_words)

In [33]:
for i, line in df.iteritems():
    line = line.lower()
    line = re.sub(r"\d+", "", line)
    line = line.translate(translate_table)
    

0 i have to say apple has by far the best customer care service i have ever received apple appstore
1 ios  is so fricking smooth  beautiful thanxapple apple
2 love u apple
3 thank you apple loving my new iphone s  apple iphones pictwittercomxmhjcupcb
4 apple has the best customer service in and out with a new phone in under min
5 apple ear pods are amazing best sound from inear headphones ive ever had
6 omg the iphone s is so cool it can read your finger print to unlock your iphone s and to make purchases without a passcode apple apple
7 the iphone c is so beautiful  apple
8 attributeownership is exactly why apple will always be one apple marketing marketer business innovation fb
9 just checked out the specs on the new ios wow is all i have to say i cant wait to get the new update  bravo apple
10 i love the new ios so much thnx apple phillydvibing
11 cant wait to get my iphones apple
12 vvista fingerprint scanner the killer feature of iphone s this is so bloody brilliant apple timesnow

739 well taught alumnart mchoto curious re apple mkt dept thought process really dont get c same  only  less no new mkt share
740  rt kemalnads jazzyjeff apple what the
741 httpthndritifdtv  phonebloks   samsung apple
742 jonjonnyp apple you will have to queue up in person with the rest of the normals  uglies to get the s
743 ha thepartycow well well motorola using twitter to fire back apple in other news go adambain pictwittercommhqfdam
744 watched someone give a presentation this morning on their apple ipad only to have their unlock pw key presses be illuminated on screen
745 yo apple where is the update to allow u to access your call history while on the phone
746 its about time that apple released 
747 silly microsoft making fun of apple wont sell more products it makes you look desperate  desperate is a bad strategy
748 apple and barackobama  laugh now but think about it pictwittercomomlzdgirs httpnblogsphle
749 apple wheres pictwittercomdnxoecpqt
750 dave nsa apple iphonec is rea